In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import glob
import os
import nibabel as nib
import numpy as np
import PIL
import matplotlib.pyplot as plt
import pdb;
import torchvision
import torch
import nibabel as nib
from torch import nn
from torch.nn import functional as F
from torchvision import models
import glob
import csv
import copy
import time
from tqdm import tqdm
#import torch
import numpy as np
import torch.utils.data as data
import torch.utils
import os
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path
import cv2
import click
import torch
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import balanced_accuracy_score
from torch.utils import data
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision import models
from pathlib import Path
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path
import torch
from PIL import Image
import PIL
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

import tensorflow as tf
from sklearn.metrics import recall_score



In [2]:
!pip install torchvision


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.device = torch.device("cuda:0")
        PATH = '/content/drive/MyDrive/Science fair 22-23//Model_log/weights.pt'
        self.fmodel = torch.load(PATH, map_location=self.device)
        self.fmodel = self.fmodel.to(torch.float32)

        self.fmodel.eval()

    def grayscale_to_rgb(self, img, num_slices=256):
        # Get shape of the images (inc batch size).
        height, width, slices = 256,256, num_slices

        # Hold RGB values.
        rgb_image = np.zeros((height, width, slices, 3), dtype=np.float32)

        # Repeat values for each channel.
        for i in range(3):
            rgb_image[..., i] = img.squeeze()

        return rgb_image

    def img_prob(self, input, batch):
        #input = torch.from_numpy(input)
        m = nn.Sigmoid()

        #batch = 4

        prob = np.zeros([batch,2])

        for batch_idx in range (0, batch):

          cur = input[batch_idx]
          no = m(cur)

          no = no.cpu().detach().numpy()

          #rewrite to make it batches
          total_pos_vals = (no>0.5).sum()
          total_neg_vals = (no<0.5).sum()

          if(total_pos_vals > 0):
            pos_val_sum = no[no>0.5].sum()
            pos_prob = pos_val_sum/total_pos_vals

          else:
            pos_prob = 0


          if (total_neg_vals >0) :
            neg_val_sum = total_neg_vals - (no[no<0.5].sum())
            neg_prob = neg_val_sum/total_neg_vals

          else:
            neg_prob = 0



          prob[batch_idx] = pos_prob,neg_prob
        #print(prob)
        return prob


    def forward(self, x):
        # input is 2, 1, 256, 256, 256
        #batch, color, height, width, slice
        x = x.cpu().detach().numpy().astype(np.float32)
        batch, color, height, width, slice_num = x.shape
        #results = np.zeros([batch, 0, height, width])
        results = np.zeros((batch, slice_num, height, width), dtype=np.float32)
        #print(results.shape)
        self.fmodel.eval()
        for batch_idx in range (0, batch):
          #cur_x shape is 1,128,128,128 (color, height, width slice)
          cur_x = x[batch_idx]

          #cur_x shape is 128,128,128,3 (height, width, slices, color)
          cur_x = self.grayscale_to_rgb(cur_x)
          #print(x.shape)

          #shape is now slice, color, height, width
          cur_x = np.transpose(cur_x, (2,3,0,1)) # reorder dimensions to match input shape
          #print(cur_x.shape)
          tensor_img = torch.from_numpy(cur_x).to(self.device)


          with torch.no_grad():
              for y in range(0, slice_num):
                  tensor_slice = tensor_img[y,:,:]

                  tensor_slice = tensor_slice.to(torch.float32)
                  # tensor_slice = tensor_slice.to(self.device)
                  # tensor_slice = tensor_slice.type(torch.cuda.FloatTensor)
                  img_t = torch.unsqueeze(tensor_slice, dim=0)
                  #output = self.fmodel(img_t)["out"]
                  #print(img_t.shape)
                  output = self.fmodel(img_t)["out"]
                  numpy_out = output.cpu().detach().numpy()[0][0]
                  #numpy_out = np.array([numpy_out[0][0]])
                  #print(numpy_out.shape)
                  #print(results[batch_idx].shape)
                  #results = np.append(results[batch_idx], numpy_out,0)
                  results[batch_idx,y,:,:] = numpy_out
                  torch.cuda.empty_cache()

        #results = torch.from_numpy(results)
        #results = torch.unsqueeze(results, dim=0)




        resized_results = np.zeros((batch, slice_num, 128,128), dtype=np.float32)
        for batch_idx in range (0, batch):
          cur_x = results[batch_idx]
          for x in range(0, slice_num):
              cur_slice = cur_x[x,:,:]
              #print(cur_slice.shape)
              resize = cv2.resize(cur_slice, dsize = (128,128), interpolation = cv2.INTER_LINEAR)
              resize = np.expand_dims(resize, axis=0) # add an extra dimension to resize
              resized_results[batch_idx,y,:,:] = resize

        resized_results = torch.from_numpy(resized_results)
        resized_results = resized_results.to(torch.float32)
        resized_results = resized_results.to(self.device)
        #print(resized_results.shape)
        #print(results.shape)

        #print(resized_results.shape)
        fin = self.img_prob(resized_results, batch = batch)
        fin = torch.from_numpy(fin)
        fin = fin.to(torch.float32)
        fin = fin.to(self.device)
        #print(fin.shape)
        return fin


  #code to load the model
  #the process nii and run through mode
  #if batching doesnt work on the processing nii, loop through each png induvidually and get a infrences
  #to compile them create an empty numpy array which is the same shape as the large nii image, for each slice, put it in the slice into the correct location
  #convert numpy to tensor and return

In [5]:
class mlp(nn.Module):
    def __init__(self):
        super(mlp, self).__init__()
        self.device = torch.device("cuda:0")
        PATH = '/content/drive/MyDrive/Science Fair 23-24/Pancreas stuff/MLP_0.8276.pt'
        self.bmodel = torch.load(PATH)
        self.bmodel.eval()

    def forward(self, y):
        output = self.bmodel(y)
        return output

In [6]:

class multimodal(nn.Module):
    def __init__(self, num_classes=2):
        super(multimodal, self).__init__()
        # Images
        #BatchNorm = nn.InstanceNorm3d
        #filters = [32, 64, 128, 256, 256, 512]
        self.backbone = MyModel()
        #self.backbone = AlignedXception(BatchNorm, filters)

        # Descriptor
        self.fc_d = nn.Linear(76, 2)  #change between 512 and 256

        # Combination

        #self._fc0 = nn.Linear(filters[-1] * 4 * 4 * 4 + 512, filters[-1])
        #self._fc0 = nn.Linear( 513, 512) #256*128*128
        #self._dropout = nn.Dropout(0.2)
        #self._fc = nn.Linear(512, num_classes)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.my_fc = nn.Linear(4,2)



    def forward(self, x, y):
        # Images
        x = self.backbone(x) #takes image goes through backbone
        #print(x.shape)


        x = x.view(x.shape[0], -1) #flattening image?
        #print(x.shape)

        # Descriptor
        y = self.sigmoid(self.fc_d(y)) #proccesses biomarker info
        #print(y.get_device())
        #print(y)
        #print(y.dtype)
        # Combination
        #print(torch.cat([x,y], dim=1))
        x = self.my_fc(torch.cat([x, y], dim=1)) #concatonates the biomarker with the image
        #print(x)
        #x = self._dropout(x) #dropout
        #x = self._fc(x) #another fully connected

        return x #gives output



    #print(output.size())
    #print(output)
    #numpy_output = output.cpu().detach().numpy()

    #plt.imshow(numpy_output)

In [7]:

class multimodalv2(nn.Module):
    def __init__(self, num_classes=2):
        super(multimodalv2, self).__init__()
        # Images
        #BatchNorm = nn.InstanceNorm3d
        #filters = [32, 64, 128, 256, 256, 512]
        self.backbone = MyModel()
        self.biomarkers = mlp()

        #self.backbone = AlignedXception(BatchNorm, filters)

        # Descriptor
        self.fc_d = nn.Linear(76, 2)  #change between 512 and 256

        # Combination

        #self._fc0 = nn.Linear(filters[-1] * 4 * 4 * 4 + 512, filters[-1])
        #self._fc0 = nn.Linear( 513, 512) #256*128*128
        #self._dropout = nn.Dropout(0.2)
        #self._fc = nn.Linear(512, num_classes)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.my_fc = nn.Linear(4,2)



    def forward(self, x, y):
        # Images
        x = self.backbone(x) #takes image goes through backbone
        #print(x.shape)


        x = x.view(x.shape[0], -1) #flattening image?
        #print(x.shape)

        # Descriptor
        y = self.sigmoid(self.biomarkers(y))
         #proccesses biomarker info
        #print(y.get_device())
        #print(y)
        #print(y.dtype)
        # Combination
        #print(torch.cat([x,y], dim=1))
        x = self.my_fc(torch.cat([x, y], dim=1)) #concatonates the biomarker with the image
        #print(x)
        #x = self._dropout(x) #dropout
        #x = self._fc(x) #another fully connected

        return x #gives output



    #print(output.size())
    #print(output)
    #numpy_output = output.cpu().detach().numpy()

    #plt.imshow(numpy_output)

In [8]:
import os
import csv
import time
import argparse
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader


In [9]:
!git clone https://github.com/BCV-Uniandes/LUCAS

import sys
sys.path.insert(0,'/content/LUCAS')

Cloning into 'LUCAS'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 40 (delta 8), reused 10 (delta 3), pack-reused 0
Receiving objects: 100% (40/40), 86.56 KiB | 805.00 KiB/s, done.
Resolving deltas: 100% (8/8), done.


In [10]:

import util.utils as utils
import util.Read_data as Read_data
from util.save_graphs import save_graph
#from modeling.model import DeepLab

In [11]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from torch.utils.data import Dataset


class MRIdataset(Dataset):
    def __init__(self, csv_file, descriptor, root_dir, image_size):
        super(MRIdataset, self).__init__()
        self.root_dir = root_dir + '/scans'
        self.image_size = np.asarray(image_size)

        self.task = -2  # -2=cancer; -1=nodule/mass

        labels = pd.read_csv(csv_file)
        self.labels = labels.set_index('patient_id').T.to_dict('list')
        descriptor = pd.read_csv(descriptor)
        self.descriptor = descriptor.set_index('patient_id').T.to_dict('list')
        self.idx = list(self.labels.keys())


        #sizes = pd.read_csv('../Data/sizes.csv')
        #sizes = sizes.set_index('access').T.to_dict('list')
        #for i in sizes:
            #if int(sizes[i][0]) < 250 and i in self.idx:
                #self.idx.remove(i)
        self.weights = self.weights_balanced()

    def __len__(self):
        return len(self.idx)

    def __getitem__(self, idx):
        patient = self.idx[idx]
        label = self.labels[patient][self.task]
        descriptor = self.descriptor[patient]
        #descriptor = descriptor[1:12] + descriptor[13:-1]
        descriptor = descriptor[1:-1]
        descriptor = np.array(descriptor, dtype=np.float32)
        #print(str(patient))
        image = load_image(str(patient) + '.nii.gz', self.root_dir)

        if any(np.asarray(image.shape) <= self.image_size):
            dif = self.image_size - image.shape
            mod = dif % 2
            dif = dif // 2
            pad = np.maximum(dif, [0, 0, 0])
            pad = tuple(zip(pad, pad + mod))
            image = np.pad(image, pad, 'reflect')

        sz = self.image_size[0]
        if any(np.asarray(image.shape) >= self.image_size):
            x, y, z = image.shape
            x = x // 2 - (sz // 2)
            y = y // 2 - (sz // 2)
            z = z // 2 - (sz // 2)
            image = image[x:x + sz, y:y + sz, z:z + sz]
        # Stats obtained from the MSD dataset
        image = np.clip(image, a_min=-1024, a_max=326)
        image = (image - 159.14433291523548) / 323.0573880113456

        return {'data': np.expand_dims(image, 0),
                'descriptor': descriptor, 'target': label, 'id':patient}

    def weights_balanced(self):
        count = [0] * 2
        for item in self.idx:
            count[self.labels[item][self.task]] += 1
        weight_per_class = [0.] * 2
        N = float(sum(count))
        for i in range(2):
            weight_per_class[i] = N/float(count[i])
        weight = [0] * len(self.idx)
        for idx, val in enumerate(self.idx):
            weight[idx] = weight_per_class[self.labels[val][self.task]]
        return weight


class collate(object):
    def __init__(self, size):
        self.transforms = DA.Compose([
            DA.NumpyToTensor()])

    def __call__(self, batch):
        elem = batch[0]
        batch = {key: np.stack([d[key] for d in batch]) for key in elem}
        return self.transforms(**batch)


def load_image(patient, root_dir):
    im = nib.load(os.path.join(root_dir, patient))
    image = im.get_fdata()
    return image

In [12]:
torch.autograd.set_detect_anomaly(True)
def main():
    lr = 0.01
    epochs = 15
    patience = 3
    batch = 4
    gpu = '1'
    training = True
    #os.environ['CUDA_VISIBLE_DEVICES'] = gpu
    image_size = [256,256,256]
    num_classes = 2
    patience = 3

    #fix
    save_path = '/content/drive/MyDrive/Science Fair 23-24/Pancreas stuff/multimodal_log' #multimodal log folder          #os.path.join('TRAIN', args.name) #change

    root = '/content/drive/MyDrive/Science fair 22-23/data/LUCAS'      # Root directory to the data
    train_file = os.path.join(root, 'new.csv')
    test_file = os.path.join(root, 'new_test.csv')
    train_des = os.path.join(root, 'train_descriptor.csv')
    test_des = '/content/drive/MyDrive/Science Fair 23-24/Pancreas stuff/test_descriptor.csv'   #os.path.join(root, 'test_descriptor.csv')


    np.random.seed(12345)
    torch.manual_seed(12345)
    torch.cuda.manual_seed_all(12345)

    model = multimodalv2(num_classes = num_classes)
    print('---> Number of params: {}'.format(
        sum([p.data.nelement() for p in model.parameters()])))



    device = torch.device("cuda:0")
    model.to(device)

    model.train()
    optimizer = optim.Adam(model.parameters(), lr = lr,
                           weight_decay=1e-5, amsgrad=True)


    annealing = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience = patience, verbose=True)
    bce = nn.BCEWithLogitsLoss()

    #dataloaders
    train_data = MRIdataset(train_file, train_des, root,
                                      image_size)
    test_data = MRIdataset(test_file, test_des, root,
                                     image_size)

    best_f1 = 0

    #try without this
    sampler = torch.utils.data.sampler.WeightedRandomSampler(
        train_data.weights, len(train_data.weights))
    train_loader = DataLoader(train_data, sampler=sampler,
                              batch_size=batch, num_workers=2)
    test_loader = DataLoader(test_data, shuffle=False, sampler=None,
                             batch_size=batch, num_workers=2)


    is_best = True
    if training:
        torch.cuda.empty_cache()
        out_file = open(os.path.join(save_path, 'progress.csv'), 'a+')

        for epoch in range(0, 15):       #epochs + 1, epochs + 1):  #double check the epoch formating and numbers mayb just hardquote
            epochs = epoch
            lr = utils.get_lr(optimizer)
            print('--------- Starting Epoch {} --> {} ---------'.format(
                epoch, time.strftime("%H:%M:%S")))
            print('Learning rate:', lr)


            #figure out what args are going into here and are used
            train_loss = train(model, train_loader, optimizer, bce)
            test_loss, f1, flag = test(model, test_loader, save_path, bce)

            out_file.write('{},{},{},{},{}\n'.format(
                epoch, train_loss, test_loss, f1, lr))
            out_file.flush()

            annealing.step(test_loss)
            save_graph(save_path)

            # To avoid saving as "the best model" one that always predict the
            # same category
            is_best = False
            if flag:
                is_best = best_f1 < f1
                best_f1 = max(best_f1, f1)

            state = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'loss': [train_loss, test_loss],
                'lr': lr,
                'f1': f1,
                'best_f1': best_f1}

            checkpoint = epoch % 50 == 0
            utils.save_epoch(state, save_path, epoch,
                             checkpoint=checkpoint, is_best=is_best)

            if lr <= (lr / (10 ** 4)):
                print('Stopping training: learning rate is too small')
                break
        out_file.close()


    #figure out what args are needed here and change that
    val_loss, flag = test(model, test_loader, save_path, bce, False)
    save_graph(save_path)





In [13]:
def train(model, loader, optimizer, bce):
    model.train()
    epoch_loss = utils.AverageMeter()
    batch_loss = utils.AverageMeter()

    device = torch.device("cuda:0")

    print_stats = len(loader) // 5
    for batch_idx, sample in enumerate(loader):
        data = sample['data'].float().to(device = device, dtype = torch.float32)
        descriptor = sample['descriptor'].to(device)
        target = sample['target'].float().to(device)
        target = torch.stack([1 - target, target], dim=1)

        optimizer.zero_grad()
        out = model(data, descriptor)
        loss = bce(out, target)

        loss.backward()
        optimizer.step()

        batch_loss.update(loss.item())
        epoch_loss.update(loss.item())

        if batch_loss.count % print_stats == 0:
            text = '{} -- [{}/{} ({:.0f}%)]\tLoss: {:.6f}'
            print(text.format(
                time.strftime("%H:%M:%S"), (batch_idx + 1),
                (len(loader)), 100. * (batch_idx + 1) / (len(loader)),
                batch_loss.avg))
            batch_loss.reset()
    print('--- Train: \tLoss: {:.6f} ---'.format(epoch_loss.avg))
    return epoch_loss.avg


def test(model, loader, save_path, bce, training=False):
    model.eval()
    epoch_loss = utils.AverageMeter()
    count, correct = 0, 0
    labels, patients, scores, predictions = [], [], [], []
    device = torch.device("cuda:0")


    for batch_idx, sample in enumerate(loader):
        data = sample['data'].float().to(device)
        descriptor = sample['descriptor'].float().to(device)
        target = sample['target'].float().to(device)
        patients.extend(sample['id'].tolist())
        labels.extend(sample['target'].tolist())

        with torch.no_grad():
            out = model(data, descriptor)
        loss = bce(out, torch.stack([1 - target, target], dim=1))
        epoch_loss.update(loss.item())

        m = nn.Sigmoid()


        confidence = F.softmax(out, dim =1)       #m(out) #mayb switch back to softmax
        #print(confidence.size())
        #print(confidence.size)
        scores.extend(confidence[:, 1].tolist())

        pred = torch.argmax(confidence, dim=1)
        predictions.extend(pred.tolist())   #THIS LINE IS BROKEN???
        count += pred.sum()
        correct += (pred * target).sum()

    print('--- Val: \tLoss: {:.6f} ---'.format(epoch_loss.avg))

    # Metrics
    roc = roc_auc_score(labels, scores)
    ap = average_precision_score(labels, scores)
    f1 = f1_score(labels, predictions)

    if not training:
        print('ROC', roc, 'AP', ap, 'F1', f1)
        rows = zip(patients, scores)
        with open(os.path.join(save_path, 'confidence.csv'), "w") as f:
            writer = csv.writer(f)
            writer.writerow(['ROC:', roc])
            writer.writerow(['AP:', ap])
            writer.writerow(['F1:', f1])
            for row in rows:
                writer.writerow(row)

    count = count.sum()
    flag = True
    if count == 0 or count == len(loader.dataset):
        flag = False
    return epoch_loss.avg, f1, flag

In [14]:
torch.cuda.empty_cache()

In [15]:
if __name__ == '__main__':
    main()

AttributeError: ignored